In [1]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Parameters
img_width, img_height = 224, 224
batch_size = 32
epochs = 50

# Directories
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Data generator for training data

# Data generator for training data with validation split
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # 20% validation split

# Separate generator for training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')  # Specify 'training' subset

# Separate generator for validation data
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')  # Specify 'validation' subset



# Data generator for test data
test_files = [os.path.join(test_dir, f) for f in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, f))]

# Create a DataFrame with the filenames
test_df = pd.DataFrame({"Filename": test_files})

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='Filename',
    y_col=None,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)


Found 7920 images belonging to 5 classes.
Found 1980 images belonging to 5 classes.
Found 100 validated image filenames.


In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dense, GlobalAveragePooling2D, Activation, Add
from tensorflow.keras.models import Model

def efficientnet_block(x, filters, kernel_size, strides):
    x = Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)  # EfficientNet uses the Swish activation function
    return x

def build_efficientnet(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)
    
    # Stem convolution
    x = Conv2D(32, 3, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    
    # Blocks
    x = efficientnet_block(x, 16, 3, strides=1)
    x = efficientnet_block(x, 24, 3, strides=2)
    x = efficientnet_block(x, 40, 5, strides=2)
    x = efficientnet_block(x, 80, 3, strides=2)
    
    # Global average pooling and classifier
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    # Create model
    model = Model(inputs, outputs)
    return model

# Define input shape and number of classes
input_shape = (224, 224, 3)  # Example input shape
num_classes = 5  # Replace with the actual number of classes

# Build the model
efficientnet_model = build_efficientnet(input_shape, num_classes)

# Compile the model
efficientnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary
efficientnet_model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_5 (Conv2D)           (None, 112, 112, 32)      896       
                                                                 
 batch_normalization_5 (Batc  (None, 112, 112, 32)     128       
 hNormalization)                                                 
                                                                 
 activation_5 (Activation)   (None, 112, 112, 32)      0         
                                                                 
 conv2d_6 (Conv2D)           (None, 112, 112, 16)      4624      
                                                                 
 batch_normalization_6 (Batc  (None, 112, 112, 16)     64        
 hNormalization)                                           

In [4]:
# Fine-tune the model
history = efficientnet_model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator
)


Epoch 1/25
248/248 [==============================] - 96s 341ms/step - loss: 1.2249 - accuracy: 0.4999 - val_loss: 1.5994 - val_accuracy: 0.2242
Epoch 2/25
248/248 [==============================] - 79s 314ms/step - loss: 1.1230 - accuracy: 0.5475 - val_loss: 1.2857 - val_accuracy: 0.4596
Epoch 3/25
248/248 [==============================] - 81s 321ms/step - loss: 1.0702 - accuracy: 0.5818 - val_loss: 1.1820 - val_accuracy: 0.5136
Epoch 4/25
248/248 [==============================] - 82s 327ms/step - loss: 1.0295 - accuracy: 0.5981 - val_loss: 1.2117 - val_accuracy: 0.5182
Epoch 5/25
248/248 [==============================] - 76s 300ms/step - loss: 1.0006 - accuracy: 0.6068 - val_loss: 1.1436 - val_accuracy: 0.5449
Epoch 6/25
248/248 [==============================] - 79s 312ms/step - loss: 0.9609 - accuracy: 0.6251 - val_loss: 1.2957 - val_accuracy: 0.5015
Epoch 7/25
248/248 [==============================] - 80s 319ms/step - loss: 0.9454 - accuracy: 0.6297 - val_loss: 1.1950 - val_ac